# 简介

无需多说，一路运行即可！

此脚本由**[托尼是塔克](https://wxp123.me)**创建，仅用于学习，请勿滥用。如有疑问，拉到最后！

In [1]:
# 查看设备，是K80还是T4,如果是K80...！
! /opt/bin/nvidia-smi

Sat Sep 21 09:17:06 2019       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 418.67       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   47C    P8    30W / 149W |      0MiB / 11441MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

# 第一步 准备好workspace
这一部你可以选着两种方式
1. 使用默认的workspace，你无需自己上传，仅用于熟悉操作。
2. 通过Google Drive （谷歌云盘）上传自己的workspace到指定目录。



谷歌云盘地址：https://drive.google.com/drive/my-drive  
注意：谷歌网址现在都需要“科学上网”才能访问。  

In [3]:
#挂载谷歌云盘
#点击链接授权，复制授权码，填入框框，然后回车。

from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
#创建DeepFaceLab目录，并且进入目录
%cd /content/drive/My Drive/
!mkdir DeepFaceLab
%cd /content/drive/My Drive/DeepFaceLab/

In [0]:
# 下面的1和2 不用同时点，二选一！！！！
# 1. 练手可以使用这一行，直接git clone一个workspace
!git clone https://github.com/dream80/DFLWorkspace.git workspace

In [0]:
# 2.正常情况肯定是通过谷歌云盘上传自己的workspace，上传前压缩一下，然后用下面的命令解压。
# 注意一定要上传到DeepFaceLab目录下。
!unzip  workspace.zip



#第二步 安装DeepFaceLab


In [0]:
# 获取DFL源代码v1.6.1稳定版

!git clone -b v1.6.1 https://github.com/dream80/DeepFaceLab_Colab.git


In [0]:
# 进入DeepFaceLab_Colab目录
%cd /content/drive/My Drive/DeepFaceLab/DeepFaceLab_Colab


In [0]:
# 安装Python依赖
!pip install -r requirements_colab.txt
!pip install --upgrade scikit-image

# 第三步. 提取脸部

In [0]:
# 确保路径正确，进入DeepFaceLab_Colab

%cd /content/drive/My Drive/DeepFaceLab/DeepFaceLab_Colab

In [0]:
#Src 视频转图片

!python main.py videoed extract-video --input-file ../workspace/data_src.mp4 --output-dir ../workspace/data_src/

In [0]:
#Src 提取脸部图片

!python main.py extract --input-dir ../workspace/data_src --output-dir ../workspace/data_src/aligned --detector s3fd --debug-dir ../workspace/data_src/aligned_debug

In [0]:
#Src排序，可以通过谷歌云盘查看结果，删除不良图片

!python main.py sort --input-dir ../workspace/data_src/aligned --by hist

In [0]:
#Dst视频转图片

!python main.py videoed extract-video --input-file ../workspace/data_dst.mp4 --output-dir ../workspace/data_dst/

In [0]:
#Dst提取脸部图片

!python main.py extract --input-dir ../workspace/data_dst --output-dir ../workspace/data_dst/aligned --detector s3fd --debug-dir ../workspace/data_dst/aligned_debug

In [0]:
#Dst排序，可以通过谷歌云盘查看结果，删除不良图片

!python main.py sort --input-dir ../workspace/data_dst/aligned --by hist

# 第四步. 训练模型


*  支持H128,SAE,DF, LIAEF128等模型，根据自己的情况选择模型。
*  训练开始是需要配置参数，记得开启预览，其他参数根据自己情况选择，使用默认参数的话直接回车即可。
*  不想训练了可以点击停止，停止时会抛出异常，但是没什么关系。下次可以继续训练

In [0]:
# 确保路径正确，进入DeepFaceLab_Colab

%cd /content/drive/My Drive/DeepFaceLab/DeepFaceLab_Colab

In [0]:
# Running trainer. H128 

!python main.py train --training-data-src-dir ../workspace/data_src/aligned --training-data-dst-dir ../workspace/data_dst/aligned --model-dir ../workspace/model --model H128 --no-preview

**模型预览方法请参考：**
https://www.wxp123.me/p/329

简单描述：左侧->文件->/driver/..../model/ *preview*.jpg->双击  
  
这种方法比在脚本中预览和在网盘预览都要方便，并且无需启动预览配置！

# 第五步. 转换输出

默认使用H128进行转换，如果需要其他模型，请修改命令中的参数。


In [0]:
#用src中的脸替换dst的脸 
!python main.py convert --input-dir ../workspace/data_dst --output-dir ../workspace/data_dst/merged --aligned-dir ../workspace/data_dst/aligned --model-dir ../workspace/model --model H128

In [0]:
#把替换好的图片转换成视频

!python main.py videoed video-from-sequence --input-dir ../workspace/data_dst/merged --output-file ../workspace/result.mp4 --reference-file ../workspace/data_dst.mp4

#第六步. 继续训练
当你第二次开始训练，或者掉线之后继续训练时并不需要执行上面所有的步骤。只需要下面简单的几个步骤。
1. 挂载云盘
2. 安装依赖
3. 开始训练

In [0]:
#挂载谷歌云盘
#点击链接授权，复制授权码，填入框框，然后回车。

from google.colab import drive
drive.mount('/content/drive', force_remount=True)

#重新覆盖解压，提升加载速度，可节省很多时间
%cd /content/drive/My Drive/DeepFaceLab/
!unzip  -o workspace.zip


# 进入DeepFaceLab_Colab目录
%cd /content/drive/My Drive/DeepFaceLab/DeepFaceLab_Colab

# 安装Python依赖
!pip install -r requirements_colab.txt
!pip install --upgrade scikit-image

# 开始训练H128 ，如果是其他模型，修改后面的参数即可，比如H128。

!python main.py train --training-data-src-dir ../workspace/data_src/aligned --training-data-dst-dir ../workspace/data_dst/aligned --model-dir ../workspace/model --model H128 --no-preview

# 工具


使用脚本练模型的时候常常会需要上传，解压，压缩，下载等，下面为大家提供一个解压和压缩的脚本。如果需要对其他文件操作，改一下路径，改一下文件名就可以了。

In [0]:
# 解压workspace.zip，假设这个文件在DeepFaceLab目录下。

%cd /content/drive/My Drive/DeepFaceLab/

#常规解压，如果之前有文件会询问是否覆盖，输入A表示覆盖
!unzip workspace.zip

#覆盖解压，直接覆盖，不需要询问。
#unzip -o workspace.zip 


In [0]:
# 压缩worksapce ，把压缩包保存在DeepFaceLab目录下
%cd /content/drive/My Drive/DeepFaceLab/
!zip -r workspace.zip  workspace

In [0]:
# 压缩data_dst
%cd /content/drive/My Drive/DeepFaceLab/workspace
!zip -r data_dst.zip  data_dst

/content/drive/My Drive/DeepFaceLab/workspace
  adding: data_dst/ (stored 0%)
  adding: data_dst/00859.png (deflated 1%)
  adding: data_dst/00860.png (deflated 0%)
  adding: data_dst/00861.png (deflated 0%)
  adding: data_dst/00862.png (deflated 0%)
  adding: data_dst/00863.png (deflated 0%)
  adding: data_dst/00864.png (deflated 0%)
  adding: data_dst/00865.png (deflated 0%)
  adding: data_dst/00866.png (deflated 0%)
  adding: data_dst/00867.png (deflated 0%)
  adding: data_dst/00868.png (deflated 0%)
  adding: data_dst/00869.png (deflated 0%)
  adding: data_dst/00870.png (deflated 0%)
  adding: data_dst/00871.png (deflated 0%)
  adding: data_dst/00872.png (deflated 0%)
  adding: data_dst/00873.png (deflated 0%)
  adding: data_dst/00874.png (deflated 0%)
  adding: data_dst/00875.png (deflated 0%)
  adding: data_dst/00876.png (deflated 0%)
  adding: data_dst/00877.png (deflated 0%)
  adding: data_dst/00878.png (deflated 0%)
  adding: data_dst/00879.png (deflated 0%)
  adding: data_dst/

# 其他

 谷歌云地址：https://drive.google.com/drive/my-drive

 作者邮箱 ：wpgdream@gmail.com
 
 使用教程：https://www.wxp123.me
 
 
不求别的，只求在github给个小星星^_^

https://github.com/dream80/DeepFaceLab_Colab

右上角star 谢谢!!!